In [ ]:
# Uninstall all conflicting packages and install new versions uniformly
!pip uninstall -y langchain langchain-core langchain-text-splitters langchain-openai langchain-community langchain-classic
# Install compatible new versions
!pip install langchain>=0.4.0
!pip install langchain-core>=1.1.0
!pip install langchain-text-splitters>=1.0.0
!pip install langchain-openai>=1.0.3
!pip install langchain-community>=0.4.1
!pip install chromadb

Found existing installation: langchain 1.1.2
Uninstalling langchain-1.1.2:
  Successfully uninstalled langchain-1.1.2
Found existing installation: langchain-core 1.1.2
Uninstalling langchain-core-1.1.2:
  Successfully uninstalled langchain-core-1.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/COMMS4705 NLP/Project'

/content/drive/MyDrive/Colab Notebooks/COMMS4705 NLP/Project


In [ ]:
import json
import os
from google.colab import userdata
CHROMA_PATH = "chroma_db"

os.environ["OPENAI_API_KEY"]=userdata.get('OPENAI_API_KEY')

os.environ['GOOGLE_GENAI_USE_VERTEXAI']='True'
os.environ['GEMINI_API_KEY']=userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN']=userdata.get('HF_TOKEN')

In [ ]:
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# ----------------- Step 3: Embedding and storage -----------------
# Use OpenAIEmbeddings to create embedding model
print("3. Creating embeddings and storing to vector database...")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
filename = "kg_crime.json"

def load_json_to_documents(file_path: str=filename):
    print(f"1. Loading JSON Lines data from file {file_path}...")

    data = []
    # --- Start: Read line by line ---
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue # Skip bad data
    # --- End ---
    documents: List[Document] = []

    # Process the loaded data list
    for item in data:
        metadata = {
            "crime_big": item.get("crime_big", "N/A"),
            "crime_small": item.get("crime_small", "N/A"),
        }

        content_parts = []
        # Process field logic
        for field in ["fatiao", "gainian", "tezheng", "chufa", "rending", "jieshi", "bianhu"]:
            if field in item and item[field]:
                if isinstance(item[field], list):
                    content_parts.append(f"--- {field} ---")
                    content_parts.extend(item[field])
                elif isinstance(item[field], str):
                    content_parts.append(f"--- {field} ---")
                    content_parts.append(item[field])

        full_content = "\n".join(content_parts)

        if full_content.strip():
            doc = Document(page_content=full_content, metadata=metadata)
            documents.append(doc)

    print(f"   Successfully loaded {len(documents)} legal document(s).")
    return documents



# Use Chroma as vector store
# Default storage in local .chroma_db folder
if not os.path.exists(CHROMA_PATH) or not os.listdir(CHROMA_PATH):

    print(f"🆕 Database not detected, creating and storing to {CHROMA_PATH}...")
    if not os.path.exists(CHROMA_PATH):
        os.makedirs(CHROMA_PATH)
    # ----------------- Step 1: Load documents -----------------
    print("1. Loading documents...")
    documents = load_json_to_documents()
    # ----------------- Step 2: Split documents -----------------
    print("2. Splitting documents...")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, # Maximum length of each chunk
        chunk_overlap=50, # Overlap length between chunks
        length_function=len
    )
    splits = text_splitter.split_documents(documents)
    # Execute time-consuming embedding operations here
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=CHROMA_PATH
    )
    print("✅ Database creation completed!")
else:
    print(f"🔄 Detected existing vector database ({CHROMA_PATH}), loading directly...")

    # Directly load existing database without embedding computation, very fast
    vectorstore = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embeddings
    )
# Set vector store as retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# ----------------- Step 4 & 5: Define RAG chain and query -----------------



# ----------------- Run query -----------------
query = "走私罪怎么判刑？"

print(f"🔍 Retrieving: {query}")
retrieved_docs = retriever.invoke(query)
# Print results
print(f"\n✅ Retrieved {len(retrieved_docs)} relevant document(s):\n")

# Combine documents into a single string
context_string = ""
for i, doc in enumerate(retrieved_docs):
    # Get filtered metadata (only crime_small and crime_big)


    # Build document string
    doc_string = f"--- Document {i+1} ---\n"
    doc_string += f"[Content]: {doc.page_content[:300]}\n"

    context_string += doc_string

print(context_string)

3. Creating embeddings and storing to vector database...
🔄 Detected existing vector database (chroma_db), loading directly...


/tmp/ipython-input-2515828311.py:82: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


🔍 Retrieving: 走私罪怎么判刑？

✅ Retrieved 20 relevant document(s):

--- Document 1 ---
[Content]: 走私淫秽物品在本条第二款规定的最高数量五倍以L，或者虽不满最高数量五倍，但具有是犯罪集团的首要分子或者使用特种车进行走私等严重情节的，属于走私淫秽物品罪“情节严重”，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。
走私非淫秽的影片、影碟、录像带、录音带、音碟、图片、书刊、电子出版物等物品的，依照刑法第一百五十三条的规定定罪处罚。推荐：更多刑法相关司法解释
--- Document 2 ---
[Content]: 走私淫秽物品在本条第二款规定的最高数量五倍以L，或者虽不满最高数量五倍，但具有是犯罪集团的首要分子或者使用特种车进行走私等严重情节的，属于走私淫秽物品罪“情节严重”，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。
走私非淫秽的影片、影碟、录像带、录音带、音碟、图片、书刊、电子出版物等物品的，依照刑法第一百五十三条的规定定罪处罚。推荐：更多刑法相关司法解释
--- Document 3 ---
[Content]: 按照该标准认定本罪后该如何处罚？
--- jieshi ---
[司法解释]
最高人民法院《关于审理走私刑事案件具体应用法律若干问题的解释》(2000.9.26法释[2000]30号)
第五条刑法第一百五十二条规定的“其他淫秽物品”，是指除淫秽的影片、录像带、录音带、图片、书刊以外的，通过文字、声音、形象等形式表现淫秽内容的影碟、音碟、电子出版物等物品。
走私淫秽物品达到下列数量之一的，属于走私淫秽物品罪“情节较轻”，处三年以下有期徒刑、拘役或者管制，并处罚金：
(一)走私淫秽录像带、影碟五十盘(张)以上至一百盘(张)的;
(二)走私淫秽录音带、音碟一百盘(张)以上至二百盘(张)的;
(三)走
--- Document 4 ---
[Content]: 按照该标准认定本罪后该如何处罚？
--- jieshi ---
[司法解释]
最高人民法院《关于审理走私刑事案件具体应用法律若干问题的解释》(2000.9.26法释[2000]30号)
第五条刑法第一百五十二条规定的“其他淫秽物品”，是指除淫秽的影片、录像带、录音带、图片、书刊以外的，通过文字、声音、

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# -------- Step 1: Load test data --------
def load_test_data(filepath, max_samples=100):
    """Load test data"""
    import json
    import random

    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                item = json.loads(line)
                fact = item.get("fact")
                accusation = item.get("meta", {}).get("accusation", [])

                if fact and accusation:
                    data.append({"question": fact, "answers": accusation})
            except:
                continue

    if len(data) > max_samples:
        random.seed(42)
        data = random.sample(data, max_samples)

    print(f"✅ Loaded {len(data)} test samples")
    return data

In [ ]:
import pandas as pd
import google.generativeai as genai
from google import genai
from google.genai import types
import torch
import os
from tqdm import tqdm
import time

def llama_compression(test_data, compress_model, tokenizer, retriever,output_file='llama_compression_15K.xlsx'):
    """Evaluate RAG + Compression (Llama)"""
    print("\n📊 Evaluating RAG + Compression (Llama)...")

    results = []
    # Ensure we verify the data slice
    samples = test_data[:]

    for i, item in enumerate(tqdm(samples, desc="RAG+Comp(Llama)")):
        try:
            question = item["question"]
            # gold = item["answers"]

            # Retrieve
            docs = retriever.invoke(question)
            context = "\n".join([doc.page_content for doc in docs])

            # Compression Prompt (Added context injection)
            comp_prompt = f"请根据以下法律内容生成简明、紧凑的摘要,只保留与回答用户问题相关的关键信息:\n\n【法律内容】:\n{context}\n\n【问题】:\n{question}\n\n摘要:"

            comp_inputs = tokenizer(comp_prompt, return_tensors="pt", truncation=True, max_length=2048).to(compress_model.device)

            with torch.no_grad():
                comp_outputs = compress_model.generate(**comp_inputs, max_new_tokens=256, do_sample=True, temperature=0.3)

            summary = tokenizer.decode(comp_outputs[0][comp_inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

            results.append({
                "id": i,
                "question": question,
                "llama_summary": summary,
                "retrieved_docs": context
            })

        except Exception as e:
            print(f"Error processing item {i} (Llama): {e}")
            results.append({
                "id": i,
                "question": question,
                "llama_summary": "Error",
                "retrieved_docs": context
            })
            continue

    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False)
    print(f"✅ Llama results saved to {output_file}")
    return df

def gemini_compression(test_data, retriever,output_file='gemini_compression_15K.xlsx'):
    """Evaluate RAG + Compression (Gemini)"""
    print("\n📊 Evaluating RAG + Compression (Gemini)...")

    # Configure Gemini
    api_key = os.environ.get('GEMINI_API_KEY')
    if not api_key:
        print("Warning: GEMINI_API_KEY not found. Please set it in os.environ.")
        return pd.DataFrame()

    # Remove incompatible genai.configure call
    # genai.configure(api_key=api_key)

    client = genai.Client(
        api_key=api_key,
        http_options=types.HttpOptions(api_version="v1")
    )

    results = []
    samples = test_data[:]

    for i, item in enumerate(tqdm(samples, desc="RAG+Comp(Gemini)")):
        try:
            # Rate limiting: Sleep 5 seconds every 20 requests
            if i > 0 and i % 20 == 0:
                time.sleep(10)

            question = item["question"]

            # Retrieve
            docs = retriever.invoke(question)
            context = "\n".join([doc.page_content for doc in docs])

            prompt = f"""请根据以下法律内容生成简明、紧凑的摘要,只保留与回答用户问题相关的关键信息。

【法律内容】:
{context}

【用户问题】:
{question}

摘要:"""

            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=prompt,
                config=types.GenerateContentConfig(
                    max_output_tokens=2000,
                    temperature=0.3,
                )
            )
            summary = response.text.strip() if response.text else ""

            results.append({
                "id": i,
                "question": question,
                "gemini_summary": summary
            })

        except Exception as e:
            print(f"Error processing item {i} (Gemini): {e}")
            results.append({
                "id": i,
                "question": question,
                "gemini_summary": f"Error: {e}"
            })

    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False)
    print(f"✅ Gemini results saved to {output_file}")
    return df


def concat_results(llama_df, gemini_df, output_file= None):
    """Merge Gemini and Llama summary results and save"""
    print("\n💾 Merging and saving results...")

    # Merge on ID
    if 'id' in llama_df.columns and 'id' in gemini_df.columns:
        merged_df = pd.merge(llama_df, gemini_df[['id', 'gemini_summary']], on='id', how='inner')
    else:
        # Fallback
        merged_df = pd.concat([llama_df, gemini_df['gemini_summary']], axis=1)

    # Save full results
    if output_file:
      merged_df.to_excel(output_file, index=False)
      print(f"✅ Full results saved to {output_file}")
    return merged_df

def main(filepath, compress_model, tokenizer, retriever, max_samples=100):
    """Main function: Load data -> Run Llama -> Run Gemini -> Merge results"""
    print(f"🚀 Starting main process (Max Samples: {max_samples})...")

    # 1. Load data (Assumes load_test_data is defined globally)
    test_data = load_test_data(filepath, max_samples=max_samples)

    # 2. Llama
    llama_df = llama_compression(test_data, compress_model, tokenizer, retriever, output_file='new_llama_compression_15K.xlsx')

    # 3. Gemini
    gemini_df = gemini_compression(test_data, retriever,output_file='new_gemini_compression_15K.xlsx')

    # 4. Merge
    final_df = concat_results(llama_df, gemini_df)

    return final_df

In [ ]:
# Fix error in gemini_compression function (remove genai.configure)

# Run main process
# Note: Please ensure previous variables (filepath, model, tokenizer, retriever) are defined
filepath = 'test.json'

# Run main function (default processing 100 samples, can modify max_samples as needed)
final_df = main(filepath, model, tokenizer, retriever, max_samples=100)

🚀 Starting main process (Max Samples: 100)...
✅ Loaded 100 test samples

📊 Evaluating RAG + Compression (Llama)...


RAG+Comp(Llama): 100%|██████████| 100/100 [1:08:58<00:00, 41.39s/it]


✅ Llama results saved to new_llama_compression_15K.xlsx

📊 Evaluating RAG + Compression (Gemini)...


RAG+Comp(Gemini):   4%|▍         | 4/100 [00:34<13:39,  8.54s/it]

Error processing item 3 (Gemini): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}


...

Error processing item 82 (Gemini): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}


RAG+Comp(Gemini):  84%|████████▍ | 84/100 [13:16<01:13,  4.61s/it]

Error processing item 83 (Gemini): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}


RAG+Comp(Gemini): 100%|██████████| 100/100 [16:14<00:00,  9.74s/it]

✅ Gemini results saved to new_gemini_compression_15K.xlsx

💾 Merging and saving results...


In [ ]:
# regenerate gemini
filepath = 'test.json'

test_data = load_test_data(filepath, max_samples=100)
gemini_df = gemini_compression(test_data, retriever,output_file='new_gemini_compression_15K.xlsx')


In [ ]:

filepath = 'test.json'

test_data = load_test_data(filepath, max_samples=100)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

gemini_df2 = gemini_compression(test_data, retriever,output_file='new_gemini_compression_20K.xlsx')


✅ Loaded 100 test samples

📊 Evaluating RAG + Compression (Gemini)...


RAG+Comp(Gemini):  60%|██████    | 60/100 [08:52<04:19,  6.48s/it]

Error processing item 59 (Gemini): 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API key expired. Please renew the API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'aiplatform.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key expired. Please renew the API key.'}]}}


RAG+Comp(Gemini): 100%|██████████| 100/100 [15:12<00:00,  9.13s/it]


✅ Gemini results saved to new_gemini_compression_20K.xlsx


In [ ]:
# Display the first few rows of results
import pandas as pd
pd.set_option('display.max_colwidth', 100)  # Set column width to view summaries
# 15K
llama_df = pd.read_excel('new_llama_compression_15K.xlsx')
gemini_df = pd.read_excel('new_gemini_compression_15K.xlsx')
trained_llama_df_K15 = pd.read_excel('trained_llama_compression_5000.xlsx', engine='openpyxl')
llama_df = concat_results(llama_df, gemini_df).copy()
trained_llama_df_K15 = concat_results(trained_llama_df_K15, gemini_df).copy()

#20K
llama_df2 = pd.read_excel('new_llama_compression_20K.xlsx')
gemini_df2 = pd.read_excel('new_gemini_compression_20K.xlsx')
trained_llama_df_K20 = pd.read_excel('k20_trained_llama_compression_5000.xlsx', engine='openpyxl')
trained_llama_df_K20 = concat_results(trained_llama_df_K20, gemini_df2).copy()
llama_df2 = concat_results(llama_df2, gemini_df2).copy()

display(llama_df.head())


💾 Merging and saving results...

💾 Merging and saving results...

💾 Merging and saving results...

💾 Merging and saving results...


,id,question,llama_summary,retrieved_docs,gemini_summary
0,0,青县人民检察院指控：\r\n自2014年1月份以来，被告人董某没有经过工商部门和食品监督部门的批准，在青县流河镇王镇店村经营上海钙骨包子铺，在加工包子的过程中违法添加含铝泡打粉。2014年11...,内容指使供货商制作、提供与实际货物不符的单证是没有事实根据的。从今天的法庭调查事实来看，当时谁都不知道这120箱是什么，可能只有陈某一个人知道要\n装什么。而且，现在的事实证明不是120箱，是...,2、卷宗中三发粮油店黄本勇、杨金龙证实，2010年4月5日，黄本勇开车送20袋面粉给张某家，10袋兴海，10袋皓香雪，地面垫上纸壳，先摞的大米，后摞的面粉，均垛在纸壳上。没有关于将没有卖出的旧...,青县人民检察院指控被告人董某未经批准经营包子铺，在加工包子时违法添加含铝泡打粉，致使面团铝残留量1220mg/kg，不符合食品安全标准。此行为可能触犯《刑法》第一百四十三条生产、销售不符合卫生...
1,1,公诉机关指控，2013年5月28日15时许，被告人郭某路过迁西县三屯营镇井泉村陈某家时，见大门开着，家中无人，产生盗窃之念，进入屋内，正在东屋翻找财物时，被回家的陈某发现抓获。公诉机关认为被告...,不能认定被告人刘X犯有诈骗罪。\n三、起诉书认定被告人刘X诈骗焦X7.2万元事实不清、证据不足。\n辩护人认为被告人刘X的行为不符合诈骗罪的犯罪构成。\n辩护人认为被告人刘X的行为不符合诈骗罪...,直接向走私人非法收购走私进口的国家非禁止进口货物、物品，数额较大的，或者在内海、领海运输、收购、贩卖国家限制进出口货物、物品，数额较大，没有合法证明的，应当适用刑法第一百五十三条的规定定罪处罚...,根据提供的法律内容，与用户问题（入户盗窃）相关的关键信息如下：\n\n1. **从犯认定及量刑原则：** 在共同犯罪中起次要或者辅助作用的，是从犯，应当从轻、减轻处罚或者免除处罚。\n2. ...
2,2,宿州市埇桥区人民检察院指控：2016年4月7日15时许，被告人赵某某来到宿州市埇桥区三八办事处新街坊小区北门女子养生会所，将被害人赵某放在会所吧台上的VIVOX6手机盗走。经物价部门鉴定，该手...,，有五十多斤重，值200块钱”。被害人汪维根陈述：“我家的狗有四五十斤，估计有50斤，是非常大的狗，值二佰多块钱”。从各被害人陈述来看，被盗狗的市场价为每斤4元左右，而价格鉴定结论书认定每斤价...,六、被告人XXX法律观念单薄，法制观念低下，根本没有经过法律教育，作为外省市来沪务工人员，属于社会弱势群体。其被经济利益所蒙蔽才走上犯罪道路，被告人是受他人的诱惑才走上犯罪道路，其在盗窃中一直...,根据用户问题和提供的法律内容，与被告人赵某某盗窃手机案相关的关键信息摘要如下：\n\n**案情概述：**\n被告人赵某某于2016年4月7日盗窃被害人赵某的VIVO X6手机一部，经鉴定价值2...
3,3,经审理查明：2015年11月7日被告人徐某通过江山市上余镇余航村第九生产队队长徐某，到同镇五程村祝某2处转包了集体的五都埂鱼塘。从2015年11月下旬开始至2016年2月期间，被告人徐某为谋利...,的供述中说，张××在干土方工程中，经常和他一起干，经常帮他解决问题。张××的供述中说，朱××在干土方工程中，经常和他一起干，经常帮他解决问题。根据刑法第六十七条的规定，“可以对被告人从轻或减轻...,三、本案情节轻微。\n被告人开采的是一个已经废弃的巷道，开采时间也仅有十来个月，而且断断续续，由于开采规模和开采时间有限，开采总量也仅有七、八百吨，销售额只有四万元左右。其情节显属轻微。以上事...,被告人徐某于2015年11月下旬至2016年2月，在承包鱼塘及周边无采矿许可手续非法开采砂石料，获利4万元。该行为造成矿产资源破坏价值24.78万元。被制止后，徐某于2016年6月28日投案自...
4,4,无锡市锡山区人民检察院起诉指控：\r\n2016年1月17日下午，被告人周某在无锡市锡山区港下花园城酒店8502房间内，趁被害人蒋某离开之际，窃得蒋某放在名片夹内的农业银行卡一张，并于次日上午...,�骗罪，关键在于其是否以非法占有为目的。本案中，被告人刘X向张XX借款一事曾在河西区人民X院进行过庭外解决，最后双方约定由被告人刘X尽快将欠款还清。至目前为止，被告人刘X已经偿还了2.5万元，...,钱是2万元(3-4页)。被告人刘某某供述，我接到刘某的电话，说他要出差，能不能解决2万元的费用，我将刘某的意思告诉黄某某、叶某某，他们两人都说没问题，黄某某当时拿了2万元钱给我，我就下楼打电话...,无锡市锡山区人民检察院指控：2016年1月17日下午，被告人周某趁被害人蒋某离开之际，在无锡市锡山区港下花园城酒店8502房间内窃取蒋某银行卡一张。次日上午，周某利用事先获知密码在港下农业银行...


In [ ]:
!pip install jieba

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
import nltk
import jieba

# final_df is already loaded and contains 'question', 'llama_summary', 'gemini_summary'


def calculate_bleu_scores(candidate_summary, gold_summary):
    """
    Calculate BLEU-1, BLEU-2, BLEU-3, BLEU-4 scores for a candidate summary against gold_summary.
    Uses jieba for tokenization.
    Raises TypeError if inputs are not strings.
    Returns a pandas Series.
    """
    # Strict type checking as requested
    if not isinstance(candidate_summary, str):
        raise TypeError(f"Candidate summary must be a string, got {type(candidate_summary)}: {candidate_summary}")
    if not isinstance(gold_summary, str):
        raise TypeError(f"Gold summary must be a string, got {type(gold_summary)}: {gold_summary}")

    if not candidate_summary.strip():
        return pd.Series({'bleu_1': 0.0, 'bleu_2': 0.0, 'bleu_3': 0.0, 'bleu_4': 0.0})

    # Use jieba for tokenization
    candidate_tokens = jieba.lcut(candidate_summary.strip())

    references = []
    if gold_summary.strip():
        references.append(jieba.lcut(gold_summary.strip()))

    if not references or not candidate_tokens:
        return pd.Series({'bleu_1': 0.0, 'bleu_2': 0.0, 'bleu_3': 0.0, 'bleu_4': 0.0})

    # Calculate BLEU with smoothing
    smoothie = SmoothingFunction().method1

    # Weights for BLEU-1 to BLEU-4
    weights = [
        (1, 0, 0, 0),
        (0.5, 0.5, 0, 0),
        (0.333, 0.333, 0.333, 0),
        (0.25, 0.25, 0.25, 0.25)
    ]

    scores = {}
    for i, weight in enumerate(weights, 1):
        try:
            scores[f'bleu_{i}'] = sentence_bleu(references, candidate_tokens, weights=weight, smoothing_function=smoothie)
        except Exception:
            scores[f'bleu_{i}'] = 0.0

    return pd.Series(scores)



/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [ ]:
print("Calculating BLEU 1-4 scores for Llama summaries K15 against Gemini summaries ...")

bleu_scores = llama_df.apply(
    lambda row: calculate_bleu_scores(row['llama_summary'], row['gemini_summary']),
    axis=1
)
print("\n--- BLEU Score Statistics for Llama K15 Summaries (vs. Gemini) ---")
bleu_scores.describe()

Calculating BLEU 1-4 scores for Llama summaries K15 against Gemini summaries ...

--- BLEU Score Statistics for Llama K15 Summaries (vs. Gemini) ---


,bleu_1,bleu_2,bleu_3,bleu_4
count,100.000000,100.000000,100.000000,100.000000
mean,0.088779,0.021682,0.008779,0.005195
std,0.040112,0.023164,0.015954,0.012746
min,0.006369,0.002021,0.001390,0.001096
25%,0.067057,0.006481,0.002847,0.001872
50%,0.086720,0.018068,0.005587,0.003087
75%,0.107160,0.032407,0.008548,0.004337
max,0.250000,0.175412,0.142138,0.122048


In [ ]:
print("Calculating BLEU 1-4 scores for trained Llama K15 summaries against Gemini summaries ...")

bleu_scores = trained_llama_df_K15.apply(
    lambda row: calculate_bleu_scores(row['summary'], row['gemini_summary']),
    axis=1
)
print("\n--- BLEU Score Statistics for trained Llama K15 Summaries (vs. Gemini) ---")
bleu_scores.describe()

Calculating BLEU 1-4 scores for trained Llama K15 summaries against Gemini summaries ...

--- BLEU Score Statistics for trained Llama K15 Summaries (vs. Gemini) ---


,bleu_1,bleu_2,bleu_3,bleu_4
count,100.000000,100.000000,100.000000,100.000000
mean,0.083716,0.019327,0.007960,0.004691
std,0.044809,0.022225,0.015094,0.012165
min,0.000000,0.000000,0.000000,0.000000
25%,0.055939,0.006024,0.002837,0.001859
50%,0.086846,0.009200,0.003788,0.002427
75%,0.106623,0.028364,0.007823,0.004053
max,0.253731,0.178091,0.142369,0.121418


In [ ]:
print("Calculating BLEU 1-4 scores for Llama summaries K20 against Gemini summaries ...")

bleu_scores = llama_df2.apply(
    lambda row: calculate_bleu_scores(row['llama_summary'], row['gemini_summary']),
    axis=1
)
print("\n--- BLEU Score Statistics for Llama Summaries K20 (vs. Gemini) ---")
bleu_scores.describe()

Calculating BLEU 1-4 scores for Llama summaries K20 against Gemini summaries ...

--- BLEU Score Statistics for Llama Summaries K20 (vs. Gemini) ---


,bleu_1,bleu_2,bleu_3,bleu_4
count,100.000000,100.000000,100.000000,100.000000
mean,0.084707,0.021695,0.009040,0.005097
std,0.043192,0.023756,0.015033,0.010994
min,0.000000,0.000000,0.000000,0.000000
25%,0.054281,0.005775,0.002657,0.001788
50%,0.076389,0.009102,0.003720,0.002356
75%,0.113523,0.032172,0.008485,0.004342
max,0.225806,0.147082,0.118153,0.099957


In [ ]:
print("Calculating BLEU 1-4 scores for trained Llama K20 summaries against Gemini summaries ...")

bleu_scores = trained_llama_df_K20.apply(
    lambda row: calculate_bleu_scores(row['summary'], row['gemini_summary']),
    axis=1
)
print("\n--- BLEU Score Statistics for trained Llama K20 Summaries (vs. Gemini) ---")
bleu_scores.describe()

Calculating BLEU 1-4 scores for trained Llama K20 summaries against Gemini summaries ...

--- BLEU Score Statistics for trained Llama K20 Summaries (vs. Gemini) ---


,bleu_1,bleu_2,bleu_3,bleu_4
count,100.000000,100.000000,100.000000,100.000000
mean,0.054615,0.013846,0.006111,0.003553
std,0.045161,0.020945,0.012688,0.009499
min,0.000000,0.000000,0.000000,0.000000
25%,0.023392,0.003742,0.002058,0.001502
50%,0.041721,0.005373,0.002567,0.001777
75%,0.075336,0.016272,0.005746,0.003269
max,0.233333,0.138592,0.112400,0.094620


In [ ]:
trained_llama_df_K15.to_excel('analysis/trained_llama_compression_15K.xlsx', index=False)
llama_df.to_excel('analysis/llama_compression_15K.xlsx', index=False)


In [ ]:
trained_llama_df_K20.to_excel('analysis/trained_llama_compression_20K.xlsx', index=False)
llama_df2.to_excel('analysis/llama_compression_20K.xlsx', index=False)